# Parallelization of IDA*

## Start Cluster

In [2]:
import ipyparallel as ipp
cluster = ipp.Cluster(engines='mpi', n=4)
cluster.start_cluster_sync()

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


<Cluster(cluster_id='1647865219-201z', profile='default', controller=<running>, engine_sets=['1647865220'])>

## View

In [3]:
rc = cluster.connect_client_sync()
rc.wait_for_engines(4)
dview = rc[:]
dview

100%|██████████| 4/4 [00:08<00:00,  2.13s/engine]


<DirectView [0, 1, 2, 3]>

## Dependencies

In [4]:
%%px
import numpy as np
import json
from datetime import datetime

## Read Data

In [5]:
%%px
def string2duration(string):
    ''' "01:50:19.3177493" to duration in seconds'''
    date =  datetime.strptime(string.split('.')[0], "%H:%M:%S")
    return date.second + 60*date.minute + 3600*date.hour

In [6]:
%%px
def read_data(path):
    global task_count
    global data
    global tasks
    file = open(path)
    data = json.load(file)
    tasks = data['nodes']
    for task in tasks:
        tasks[task]['Data'] = string2duration(tasks[task]['Data'])
    task_count = len(tasks)

## Node Class

In [7]:
%%px
class Node():
    def __init__(self, id = '', cost = 0):
        self.children = []
        self.parents = []
        self.id = id
        self.cost = cost
    
    def successors(self):
        return sorted(self.children, key=lambda n: h(n))
    
    def is_goal(self):
        return self.id == 'GOAL'

## Graph Class

In [8]:
%%px
class Graph():
    def __init__(self, graph):
        self.nodes = {}
        self.roots = []
        self.load(graph)
        print(list(self.nodes)[-1])
    
    def load(self, graph):
        for id in graph.keys():
            try:
                node = self.nodes[id]
            except:
                node = Node(id, graph[id]['Data'])
            
            if len(graph[id]['Dependencies']) == 0:
                node.isTerminal = False
                self.roots.append(node)
            else:
                for parent in graph[id]['Dependencies']:
                    try:
                        parent_node = self.nodes[str(parent)]
                    except:
                        self.nodes[str(parent)] = Node(str(parent), graph[str(parent)]['Data'])
                        parent_node = self.nodes[str(parent)]
                        
                    parent_node.children.append(node)
                    node.parents.append(parent_node)

            self.nodes[id] = node

## Heuristic

In [9]:
%%px
def h(n):
    return 1

## Search

In [10]:
%%px
def search(path, g, bound, goal):
    node = path[-1]
    f = g + h(node)
    if f > bound:
        return f
    if node.id == goal:
        return [node.id]
    minimum = np.inf
    for succ in node.successors():
        if succ not in path:
            path.append(succ)
            t = search(path, g + succ.cost, bound, goal)
            if isinstance(t, list):
                return [node.id] + t
            elif t < minimum:
                minimum = t
            path.pop()
    return minimum

## IDA*

In [11]:
%%px
def ida_star(root, goal):
    bound = np.inf
    path = [root]
    t = search(path, root.cost, bound, goal)
    if isinstance(t, list):
        return t
    return 'Not found'

## Load Data

In [12]:
%%px
read_data("Graphs/smallComplex.json")

## Load Graph

In [13]:
%%px
graph = Graph(tasks)

[stdout:1] 3538


[stdout:3] 3538


[stdout:2] 3538


[stdout:0] 3538


## Launch IDA*

In [14]:
%%px
path = ida_star(graph.roots[0], '3538')
print(path)

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stdout:0] ['2365', '3532', '3538']


[stdout:1] ['2365', '3532', '3538']


[stdout:3] ['2365', '3532', '3538']


[stdout:2] ['2365', '3532', '3538']


%px: 100%|██████████| 4/4 [00:00<00:00, 38.04tasks/s]


## Shutdown Cluster

In [15]:
await cluster.stop_cluster()

Stopping controller
Failed to remove C:\Users\Oussama Janaheddine\.ipython\profile_default\log\ipcontroller-1647865219-201z-12692.log: [WinError 2] Le fichier spécifié est introuvable: 'C:\\Users\\Oussama Janaheddine\\.ipython\\profile_default\\log\\ipcontroller-1647865219-201z-12692.log'
Controller stopped: {'exit_code': 1, 'pid': 8552, 'identifier': 'ipcontroller-1647865219-201z-12692'}
Stopping engine(s): 1647865220
Failed to remove C:\Users\Oussama Janaheddine\.ipython\profile_default\log\ipengine-1647865219-201z-1647865220-12692.log: [WinError 32] Le processus ne peut pas accéder au fichier car ce fichier est utilisé par un autre processus: 'C:\\Users\\Oussama Janaheddine\\.ipython\\profile_default\\log\\ipengine-1647865219-201z-1647865220-12692.log'
Failed to remove C:\Users\Oussama Janaheddine\.ipython\profile_default\log\ipengine-1647865219-201z-1647865220-12692.log: [WinError 32] Le processus ne peut pas accéder au fichier car ce fichier est utilisé par un autre processus: 'C: